# Project Overview: 
## How are the top 50 companies in the S&P 500 connected through shared board members?

For my analysis, I looked at the S&P's 50 largest companies. I can pull data from Reuters based on company stock tickers. I scraped these tickers from https://www.slickcharts.com/sp500 on July 31, 2019, 
which lists S&P 500 component weights from largest to smallest.

Note: Alphabet (AKA Google) has two stock tickers: GOOG and GOOGL (GOOG shares have no voting rights, while GOOGL shares do). Since both classes of shares have the same board members, I eliminated the ticker GOOG from my list of tickers.



In [3]:
import pandas as pd
from bs4 import BeautifulSoupimport requests

In [64]:
tickers = []
names = []

URL = "https://www.slickcharts.com/sp500" 
response = requests.get(URL) 
soup = BeautifulSoup(response.content) 

table = soup.find('tbody')

count = 0
rows = table.find_all('tr')
for row in rows:
    count = count + 1
    if count == 52:
        break 
    else:
        ticker = (f"{row.find_all('td')[2].text}")
        name = (f"{row.find_all('td')[1].text}")
        names.append(name)
        tickers.append(ticker) 

'MSFT': 'Microsoft Corporation',
'AAPL': 'Apple Inc.',
'AMZN': 'Amazon.com Inc.',
'FB': 'Facebook Inc. Class A',
'BRK.B': 'Berkshire Hathaway Inc. Class B',
'JPM': 'JPMorgan Chase & Co.',
'GOOG': 'Alphabet Inc. Class C',
'GOOGL': 'Alphabet Inc. Class A',
'JNJ': 'Johnson & Johnson',
'XOM': 'Exxon Mobil Corporation',
'V': 'Visa Inc. Class A',
'PG': 'Procter & Gamble Company',
'BAC': 'Bank of America Corp',
'DIS': 'Walt Disney Company',
'T': 'AT&T Inc.',
'MA': 'Mastercard Incorporated Class A',
'CSCO': 'Cisco Systems Inc.',
'UNH': 'UnitedHealth Group Incorporated',
'HD': 'Home Depot Inc.',
'CVX': 'Chevron Corporation',
'VZ': 'Verizon Communications Inc.',
'INTC': 'Intel Corporation',
'PFE': 'Pfizer Inc.',
'MRK': 'Merck & Co. Inc.',
'KO': 'Coca-Cola Company',
'WFC': 'Wells Fargo & Company',
'CMCSA': 'Comcast Corporation Class A',
'PEP': 'PepsiCo Inc.',
'BA': 'Boeing Company',
'C': 'Citigroup Inc.',
'MCD': 'McDonald's Corporation',
'WMT': 'Walmart Inc.',
'ABT': 'Abbott Laboratories',
'ADBE'

In [33]:
len(tickers)

51

In [58]:
names


['Microsoft Corporation',
 'Apple Inc.',
 'Amazon.com Inc.',
 'Facebook Inc. Class A',
 'Berkshire Hathaway Inc. Class B',
 'JPMorgan Chase & Co.',
 'Alphabet Inc. Class C',
 'Alphabet Inc. Class A',
 'Johnson & Johnson',
 'Exxon Mobil Corporation',
 'Visa Inc. Class A',
 'Procter & Gamble Company',
 'Bank of America Corp',
 'Walt Disney Company',
 'AT&T Inc.',
 'Mastercard Incorporated Class A',
 'Cisco Systems Inc.',
 'UnitedHealth Group Incorporated',
 'Home Depot Inc.',
 'Chevron Corporation',
 'Verizon Communications Inc.',
 'Intel Corporation',
 'Pfizer Inc.',
 'Merck & Co. Inc.',
 'Coca-Cola Company',
 'Wells Fargo & Company',
 'Comcast Corporation Class A',
 'PepsiCo Inc.',
 'Boeing Company',
 'Citigroup Inc.',
 "McDonald's Corporation",
 'Walmart Inc.',
 'Abbott Laboratories',
 'Adobe Inc.',
 'Netflix Inc.',
 'Medtronic Plc',
 'Oracle Corporation',
 'International Business Machines Corporation',
 'Philip Morris International Inc.',
 'PayPal Holdings Inc',
 'Union Pacific Corpo

In [42]:
tickers.remove('GOOG')

In [41]:
len(tickers)

50

Now, it's time to scrape Reuters. Reuters.com lists company officers associated with various stock tickers.

In [43]:
companies = []
for ticker in tickers:
    try:
        URL = f"https://www.reuters.com/finance/stocks/company-officers/{ticker}" 
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content)
        rows = soup.find('tbody', class_="dataSmall").find_all('tr')
        for row in rows:
            people = {}
            try:
                people['Name'] = (row.find_all('td')[0].text.strip())
                people['Age'] = (row.find_all('td')[1].text.strip())
                people['Since'] = (row.find_all('td')[2].text.strip())
                people['Position'] = (row.find_all('td')[3].text.strip())
                people['Company'] = ticker
            except:
                pass

            companies.append(people)

    except:
        print(URL, "is broken.")


In [44]:
df = pd.DataFrame(companies)

In [45]:
df = df.dropna()

In [46]:
df.to_csv("Reuters_scrape_master.csv")

I also want to create a dataframe that simply lists each company and its associated officers.

In [47]:
master = []
for ticker in tickers:
    try:
        URL = f"https://www.reuters.com/finance/stocks/company-officers/{ticker}" 
        response = requests.get(URL) 
        soup = BeautifulSoup(response.content)
        rows = soup.find('tbody', class_="dataSmall").find_all('tr')
        companies = {}
        count = 0
        for row in rows:
            try:
                count = count + 1
                person = row.find_all('td')[0].text.strip()
                person = person.replace(u'\xa0', u' ')
                companies[count] = person
            except:
                pass
        master.append(companies)
        count = 0 
    except:
        print(URL, "is broken.")
        

In [48]:
df2 = pd.DataFrame(master)

In [49]:
df2.index = tickers

In [52]:
df2.head()

,2,3,4,5,6,7,8,9,10,11,...,27,28,29,30,31,32,33,34,35,36
MSFT,John Thompson,Bradford Smith,Satya Nadella,William Gates,Amy Hood,Kevin Scott,Christopher Capossela,Kathleen Hogan,Jean-Philippe Courtois,Margaret Johnson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAPL,Art Levinson,Timothy Cook,Luca Maestri,Jeffrey Williams,Katherine Adams,Philip Schiller,Eddy Cue,Craig Federighi,John Giannandrea,Deirdre O'Brien,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AMZN,Jeffrey Bezos,Brian Olsavsky,Andrew Jassy,Jeffrey Wilke,David Zapolsky,Jeffrey Blackburn,Shelley Reynolds,Rosalind Brewer,Jamie Gorelick,Daniel Huttenlocher,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FB,Mark Zuckerberg,David Wehner,Sheryl Sandberg,Michael Schroepfer,Colin Stretch,David Fischer,Susan Desmond-Hellmann,Marc Andreessen,Kenneth Chenault,Peter Thiel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BRK.B,Warren Buffett,Charles Munger,Marc Hamburg,Gregory Abel,Ajit Jain,Howard Buffett,Ronald Olson,Stephen Burke,Susan Decker,William Gates,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
df2.to_csv("Reuters_scrape_people.csv")

Scraping S&P 500 stocks by GICS Sector for colour-coding later

In [78]:
sectors = []
URL = f"https://en.wikipedia.org/wiki/List_of_S%26P_500_companies" 
response = requests.get(URL) 
soup = BeautifulSoup(response.content)
rows = soup.find('tbody').find_all('tr')
for row in rows:
    try:
        data = {}
        ticker=(row.find_all('td')[0].text.strip())
        data['sector']=(row.find_all('td')[3].text.strip())
        sectors.append(data)
    except:
        continue


In [119]:
sector_dict = []
for companies in sectors:
    for key, value in companies.items():
        for ticker in tickers:
            if key == ticker:
                sector_dict.append(companies)
    
sector_dict
        

[{'ABT': 'Health Care'},
 {'ACN': 'Information Technology'},
 {'ADBE': 'Information Technology'},
 {'GOOGL': 'Communication Services'},
 {'GOOG': 'Communication Services'},
 {'AMZN': 'Consumer Discretionary'},
 {'AMGN': 'Health Care'},
 {'AAPL': 'Information Technology'},
 {'T': 'Communication Services'},
 {'BAC': 'Financials'},
 {'BRK.B': 'Financials'},
 {'BA': 'Industrials'},
 {'AVGO': 'Information Technology'},
 {'CVX': 'Energy'},
 {'CSCO': 'Information Technology'},
 {'C': 'Financials'},
 {'KO': 'Consumer Staples'},
 {'CMCSA': 'Communication Services'},
 {'COST': 'Consumer Staples'},
 {'XOM': 'Energy'},
 {'FB': 'Communication Services'},
 {'HD': 'Consumer Discretionary'},
 {'HON': 'Industrials'},
 {'INTC': 'Information Technology'},
 {'IBM': 'Information Technology'},
 {'JNJ': 'Health Care'},
 {'JPM': 'Financials'},
 {'MA': 'Information Technology'},
 {'MCD': 'Consumer Discretionary'},
 {'MDT': 'Health Care'},
 {'MRK': 'Health Care'},
 {'MSFT': 'Information Technology'},
 {'NFLX': 

In [128]:
result = {}
for d in sector_dict:
    result.update(d)

In [129]:
result

{'ABT': 'Health Care',
 'ACN': 'Information Technology',
 'ADBE': 'Information Technology',
 'GOOGL': 'Communication Services',
 'GOOG': 'Communication Services',
 'AMZN': 'Consumer Discretionary',
 'AMGN': 'Health Care',
 'AAPL': 'Information Technology',
 'T': 'Communication Services',
 'BAC': 'Financials',
 'BRK.B': 'Financials',
 'BA': 'Industrials',
 'AVGO': 'Information Technology',
 'CVX': 'Energy',
 'CSCO': 'Information Technology',
 'C': 'Financials',
 'KO': 'Consumer Staples',
 'CMCSA': 'Communication Services',
 'COST': 'Consumer Staples',
 'XOM': 'Energy',
 'FB': 'Communication Services',
 'HD': 'Consumer Discretionary',
 'HON': 'Industrials',
 'INTC': 'Information Technology',
 'IBM': 'Information Technology',
 'JNJ': 'Health Care',
 'JPM': 'Financials',
 'MA': 'Information Technology',
 'MCD': 'Consumer Discretionary',
 'MDT': 'Health Care',
 'MRK': 'Health Care',
 'MSFT': 'Information Technology',
 'NFLX': 'Communication Services',
 'ORCL': 'Information Technology',
 'PY

In [215]:
df3 = df2

In [216]:
df3['sector'] = df3.index

In [217]:
df3 = df3['sector'].reset_index()

In [218]:
df3 = df3.replace({"sector": result})

In [219]:
df3 = df3.rename(columns={'index':'ticker'})

In [220]:
df3['sector'].value_counts()

Information Technology    14
Health Care                8
Communication Services     7
Consumer Staples           6
Financials                 5
Industrials                4
Consumer Discretionary     4
Energy                     2
Name: sector, dtype: int64

In [221]:
di = {
'Information Technology':1,
'Health Care':2,
'Communication Services':3,
'Consumer Staples':4,
'Financials':5,
'Industrials':6,
'Consumer Discretionary':7,
'Energy':8
}

In [222]:
df3 = df3.replace({"sector": di})

In [223]:
df3.head()

,ticker,sector
0,MSFT,1
1,AAPL,1
2,AMZN,7
3,FB,3
4,BRK.B,5


In [240]:
df3 = df3[(df3.ticker != 'PM') & (df3.ticker !='UNP') & (df3.ticker != 'TXN')]


In [242]:
df3.to_csv("stock_sectors.csv", index=False)